### Getting started:

##### Connection Setup:
   
   Connect two Supernovas between them via their I3C High Voltage connectors, and connect also each one to the computer via the USB port. One will act as an I3C controller and the other one as an I3C target.

   The target Supernova acts as a 2 bytes-addressable and non-linear memory of 1024 bytes. The memory can be accessed via USB when using the `i3cTargetWriteMemory` or `i3cTargetReadMemory` methods. Or it can be accessed through the I3C bus when the user requests an I3C write, I3C read or I3C write-read to the Supernova acting as the I3C controller addressing the Supernova acting as a target.

##### Expected Messages and Notifications:

   Both Supernovas will return a response to their requests. However, when a request is sent to the Supernova acting as a the controller to issue an I3C transfer, whether it is a private transfer or a CCC, the Supernova controller will send a response to the USB Host (in this case, this notebook), but the Supernova acting as a target will send also an asynchronous notification to notify the I3C bus event that occurred.
   
##### Testing Process:
   - Run each cell in order.
   - Look for immediate feedback:
     - The response or the notification and response for each command directed to a Supernova.


### 1. Startup

#### 1.1 Import the python package

In [1]:
import BinhoSupernova
from BinhoSupernova.Supernova import Supernova
from BinhoSupernova.commands.i3c.definitions import *

import random
import threading

print_lock = threading.Lock()

#### 1.2 List all the Supernova devices connected to the PC host

The ``BinhoSupernova.getConnectedSupernovaDevicesList()`` returns a list of the Supernova devices plugged into the host PC machine.

Two different Supernovas with different serial numbers and paths must appear in the response. Their paths are get and saved in global variables to be used later in the code to open connection.

In [ ]:
connected_devices = BinhoSupernova.getConnectedSupernovaDevicesList()
print(connected_devices)

if len(connected_devices) < 2:
    print("Two Supernovas connected to this PC are required")

In [ ]:
# Get the USB path of both devices.
I3C_TARGET_USB_PATH = connected_devices[0]["path"]
I3C_CONTROLLER_USB_PATH = connected_devices[1]["path"]

#### 1.3 Define a function to generate transaction IDs

All the request messages sent to the Supernova from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[1, 65535]``. The id value `0` is reserved for asynchronous notifications.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [3]:
# Auxiliar code to generate IDs

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

#### 1.4 Define and register callbacks to handle responses and notifications from the two Supernovas (the one acting as an I3C target and the one acting as an I3C controller)

To handle responses and notifications from Supernova, a callback function must be defined and registered. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(supernova_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Supernova.onEvent(callback_function)`` method.

In [4]:
shared_controller_response = [0]

# Define callback function
def controller_callback_function(supernova_message: dict, system_message: dict) -> None:

    with print_lock:

        if supernova_message != None:

            # Print a header
            print(">> New message from Supernova I3C Controller:")
            shared_controller_response[0] = supernova_message
            print(supernova_message)

        if system_message != None:

            # Print a header
            print(">> New message from the SYSTEM:")
            print(system_message)

In [5]:
# Define callback function
def target_callback_function(supernova_message: dict, system_message: dict) -> None:

    with print_lock:

        if supernova_message != None:

            # Print a header
            print(">> New message from Supernova I3C Target:")
            print(supernova_message)

        if system_message != None:

            # Print a header
            print(">> New message from the SYSTEM:")
            print(system_message)

### 2. Configure the Supernova that will act as an I3C target

#### 2.1 Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [7]:
supernova_target = Supernova()

#### 2.2 Open connection to the Supernova that acts as a target using the path of one of the devices retrieved by getConnectedSupernovaDevicesList() 

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(vid, pid, serial, path)
```

- ``vid: int``: The Supernova USB VID, which is set by default.
- ``pid: int``: The Supernova USB PID, which is set by default.
- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``BinhoSupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.

In [ ]:
supernova_target.open(path = I3C_TARGET_USB_PATH)

#### 2.3 Register the callback to the I3C target

In [ ]:
supernova_target.onEvent(target_callback_function)

#### 2.4 Initialize the Supernova I3C peripheral as a target memory of 1024 registers of 1 byte data width

In [ ]:
memory_layout = I3cTargetMemoryLayout_t.MEMORY_1_BYTE_X_1024_REGS

pid = [0x01, 0x02, 0x03, 0x04, 0x05, 0x06]

bcr = I3cBcrDeviceRoleBits_t.I3C_TARGET.value | \
      I3cBcrAdvancedCapabilitiesBit_t.NOT_SUPPORTED.value | \
      I3cBcrVirtualSupportBit_t.NOT_SUPPORTED.value | \
      I3cBcrOfflineCapBit_t.OFFLINE_UNCAPABLE.value | \
      I3cBcrIbiPayloadBit_t.IBI_WITHOUT_PAYLOAD.value | \
      I3cBcrIbiCapableBit_t.NOT_IBI_CAPABLE.value | \
      I3cBcrMaxDataSpeedLimitBit_t.NO_DATA_SPEED_LIMIT.value
print("BCR: ", bcr)

dcr = I3cTargetDcr_t.I3C_TARGET_MEMORY
print("DCR: ", dcr)

static_address = 0x21

mwl = 255
mrl = 255

request_result = supernova_target.i3cTargetInit(getId(), memory_layout, pid, bcr, dcr, static_address, mwl, mrl)

#### 2.5 Take a look at the internal memory

The public method ``Supernova.i3cTargetReadMemory()`` allows the USB Host to read data from the Supernova internal memory.

```python
i3cTargetReadMemory(id, memoryAddress, length)
```

- ``id: c_uint16``: 2-bytes integer from 1 to 65535 that identifies the command.
- ``memoryAddress: c_uint16``: Address of the memory at which data will be read from.
- ``length: c_uint16``: Length of the data to be read. From 1 to 1024 bytes.

In [ ]:
supernova_target.i3cTargetReadMemory(getId(),0x0000, 15)

#### 2.6 Write data into the internal memory

The public method ``Supernova.i3cTargetWriteMemory()`` allows the USB Host to send and write data into the Supernova internal memory.
```python
i3cTargetWriteMemory(id, memoryAddress, data)
```

- ``id: c_uint16``: Identifies the command.
- ``memoryAddress: c_uint16``: Address of the memory at which data will be written from.
- ``data: list``: Data to be written into the Supernova memory.

In [ ]:
data = [i for i in range(10,0,-1)]

supernova_target.i3cTargetWriteMemory(getId(), 0x0000, data)

In [ ]:
supernova_target.i3cTargetReadMemory(getId(),0x0000, 15)

#### 2.7 Data length vs available space

In [ ]:
memory_address = random.randint(0,1023)
available_space = 1024 - memory_address

data_length = random.randint(1,1024)
data_to_write = random.choices(range(0,256), k=data_length)

print(f"Random memory address: {memory_address}")
print(f"Available space: {available_space}")
print(f"Create list of random numbers of length {data_length}.")

if available_space >= data_length:
    print("The result will be: SUCCESS")
else:
    print("The result will be: SURPASSED MEMORY SIZE")

print()
print("Supernova response:")
print()

request_result = supernova_target.i3cTargetWriteMemory(getId(), memory_address, data_to_write)

In [ ]:
request_result = supernova_target.i3cTargetReadMemory(getId(), memory_address, data_length)

In [ ]:
if shared_target_response["payload"] == data_to_write[:shared_target_response["payload_length"]]:
    print("Data transferred successfully!")

### 3. Configure the Supernova that will act as an I3C controller

#### 3.1 Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [15]:
supernova_controller = Supernova()

#### 3.2 Open connection to the Supernova that acts as a controller using the path of one of the devices retrieved by getConnectedSupernovaDevicesList() 

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(vid, pid, serial, path)
```

- ``vid: int``: The Supernova USB VID, which is set by default.
- ``pid: int``: The Supernova USB PID, which is set by default.
- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``BinhoSupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.

In [ ]:
supernova_controller.open(path = I3C_CONTROLLER_USB_PATH)

#### 3.3 Register the callback to the I3C controller

In [ ]:
supernova_controller.onEvent(controller_callback_function)

#### 3.4 Initialize the Supernova I3C peripheral as controller

``Supernova.i3cControllerInit(id, pushPullRate, i3cOpenDrainRate, i2cOpenDrainRate)``

This method initializes the Supernova as an I3C controller.

  - `id: int`: A 2-byte integer that represents the transfer ID.
  - `pushPullRate: I3cPushPullTransferRate`: Push-Pull frequency.
  - `i3cOpenDrainRate: I3cOpenDrainTransferRate`: I3C frequency in Open Drain mode.
  - `i2cOpenDrainRate: I2cTransferRate`: I2C frequency.

In [ ]:
request_result = supernova_controller.i3cControllerInit(getId(), I3cPushPullTransferRate.PUSH_PULL_5_MHZ, I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ, I2cTransferRate._100KHz)

#### 3.5 Set I3C bus voltage

By default, both Supernovas work at 1.62V. If it is necessary to change the bus voltage, the voltage must be set in both devices.

In [ ]:
#Uncomment lines below if it is required to change the I3C bus voltage.

# Set the voltage of the Supernova acting as a controller.
supernova_controller.setI3cVoltage(getId(), 3300)

# In the case of the target, use one of the following methods.
# supernova_target.setI3cVoltage(getId(), 3300)
supernova_target.useExternalI3cVoltage(getId())

#### 3.6 I3C bus initialization

In [ ]:
request_result = supernova_controller.i3cControllerResetBus(getId())

In [ ]:
dynamic_address = 0x73

request_result = supernova_controller.i3cSETDASA(getId(), static_address, dynamic_address)

In [ ]:
request_result = supernova_controller.i3cControllerGetTargetDevicesTable(getId())

#### 3.7 I3C CCC Transfers

##### 3.7.1 GETPID

In [ ]:
TARGET_ADDRESS  = dynamic_address

request_result = supernova_controller.i3cGETPID(getId(), TARGET_ADDRESS)

##### 3.7.2 GETBCR

In [ ]:
TARGET_ADDRESS  = dynamic_address

request_result = supernova_controller.i3cGETBCR(getId(), TARGET_ADDRESS)

##### 3.7.3 GETDCR

In [ ]:
TARGET_ADDRESS  = dynamic_address

request_result = supernova_controller.i3cGETDCR(getId(), TARGET_ADDRESS)

In [ ]:
# Check DCR value meaning.
I3cTargetDcr_t(shared_controller_response[0]["payload"][0]).name

##### 3.7.4 GETMWL

In [ ]:
TARGET_ADDRESS  = dynamic_address

request_result = supernova_controller.i3cGETMWL(getId(), TARGET_ADDRESS)

##### 3.7.5 SETMWL

In [ ]:
TARGET_ADDRESS  = dynamic_address

request_result = supernova_controller.i3cGETMWL(getId(), TARGET_ADDRESS)

##### 3.7.6 GETMWL

In [ ]:
TARGET_ADDRESS  = dynamic_address

request_result = supernova_controller.i3cGETMWL(getId(), TARGET_ADDRESS)

##### 3.7.7 GETMRL

In [ ]:
TARGET_ADDRESS  = dynamic_address

request_result = supernova_controller.i3cGETMWL(getId(), TARGET_ADDRESS)

##### 3.7.8 SETMRL

In [ ]:
TARGET_ADDRESS  = dynamic_address
NEW_MRL         = 512

request_result = supernova_controller.i3cDirectSETMRL(getId(), TARGET_ADDRESS, NEW_MRL)

##### 3.7.9 GETMRL

In [ ]:
TARGET_ADDRESS  = dynamic_address

request_result = supernova_controller.i3cGETMRL(getId(), TARGET_ADDRESS)

##### 3.7.10 GETSTATUS

In [ ]:
TARGET_ADDRESS  = dynamic_address

request_result = supernova_controller.i3cGETSTATUS(getId(), TARGET_ADDRESS)

#### 3.8 Set I3C Target parameters

The public method ``Supernova.i3cTargetSetParameters()`` sets the configuration of the supernova acting as a target on the I3C bus.

In [ ]:
memory_layout = I3cTargetMemoryLayout_t.MEMORY_1_BYTE_X_1024_REGS

mwl = 32
mrl = 64

request_result = supernova_target.i3cTargetSetParameters(getId(), memory_layout, mwl, mrl)

3.8.1 Check the correct setting of the MWL using the method `i3cTargetSetConfiguration` above by requesting a GETMWL CCC.

In [ ]:
TARGET_ADDRESS  = dynamic_address

request_result = supernova_controller.i3cGETMWL(getId(), TARGET_ADDRESS)

3.8.2 Check the correct setting of the MRL using the method `i3cTargetSetConfiguration` above by requesting a GETMRL CCC.

In [ ]:
TARGET_ADDRESS  = dynamic_address

request_result = supernova_controller.i3cGETMRL(getId(), TARGET_ADDRESS)

#### 3.9 I3C Private Transfers

##### 3.9.1 I3C TARGET WRITE MEMORY

The public method ``Supernova.i3cTargetWriteMemory()`` writes the memory of the target Supernova using only USB communication.

```python
i3cTargetWriteMemory(id, memoryAddr, data)
```

- ``id: c_uint16``: Identifies the command.
- ``memoryAddr: c_uint16``: Address of the memory to start writing.
- ``data: list``: Data to write.

In [ ]:
MEMORY_ADDRESS  = 0x0000
DATA            = [i%0xFA for i in range(0,1024)]

request_result = supernova_target.i3cTargetWriteMemory(getId(), MEMORY_ADDRESS, DATA)

##### 3.9.2 I3C TARGET READ MEMORY

The public method ``Supernova.i3cTargetReadMemory()`` reads the memory of the target Supernova using only USB communication.

```python
i3cTargetReadMemory(id, memoryAddr, length)
```

- ``id: c_uint16``: Identifies the command.
- ``memoryAddr: c_uint16``: Address of the memory to start reading.
- ``length: c_uint16``: Data length the user intends to read.

In [ ]:
MEMORY_ADDRESS  = 0x0000
DATA_LENGTH     = 1024 # in bytes

request_result = supernova_target.i3cTargetReadMemory(getId(), MEMORY_ADDRESS, DATA_LENGTH)

#### 3.10 I3C WRITE AND READ

##### 3.10.1 I3C WRITE INDICATING THE START ADDRESS AND THE DATA

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x00, 0x00]
DATA            = [0xEE for i in range(5)]

request_result = supernova_controller.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

##### 3.10.2 I3C READ WITHOUT INDICATING THE START ADDRESS

The target will assume that the start of the new operation is the end of the previous one

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = []
LENGTH          = 10

request_result = supernova_controller.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

##### 3.10.3 I3C WRITE-READ

A write followed by a read with a RS in between

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0xBC, 0x02] # 700
LENGTH          = 5 # in bytes

request_result = supernova_controller.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

##### 3.10.4 I3C WRITE INDICATING THE START ADDRESS FOLLOWED BY A READ WITHOUT ADDRESS

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0xF4, 0x03] # 1012
DATA            = []

request_result = supernova_controller.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = []
LENGTH          = 6

request_result = supernova_controller.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

#### 3.10.5 BORDER CASES

##### 3.10.5.1 USER TRIES TO START A TRANSFER SURPASSING THE MEMORY RANGE

The target will ignore the bytes of address and will start the transfer from the end of the the previous one.

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x01, 0x04] # 1025
DATA            = [0xEE for i in range(4)]

request_result = supernova_controller.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

##### 3.10.5.2 THE TRANSFER STARTS IN AN ALLOWED ADDRESS BUT TRIES TO SURPASS THE MEMORY RANGE ON THE GO

It will only modify the bytes in the allowed range and discard the rest. The end of the transfer is taken as the end of the memory

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0xFC, 0x03] # 1020
DATA            = []

request_result = supernova_controller.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = []
LENGTH          = 30

request_result = supernova_controller.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

#### 3.11 Initialize the Supernova I3C peripheral as a target memory with a register size of 4 bytes

In [ ]:
MICRO_SECONDS_TO_WAIT_FOR_IBI = 0x69
MAX_READ_LENGTH = 0x100
MAX_WRITE_LENGTH = 0x100

features =  DdrOk.ALLOWED_DDR.value |   \
            IgnoreTE0TE1Errors.IGNORE_ERRORS.value |   \
            MatchStartStop.NOT_MATCH.value |   \
            AlwaysNack.NOT_ALWAYS_NACK.value 

request_result = supernova_target.i3cTargetInit(getId(), I3cTargetMemoryLayout_t.MEM_4_BYTES, MICRO_SECONDS_TO_WAIT_FOR_IBI, MAX_READ_LENGTH, MAX_WRITE_LENGTH, features)

#### 3.12 Init the bus

In [ ]:
request_result = supernova_controller.i3cInitBus(getId())

#### 3.13 I3C TARGET MEMORY RELATED COMMANDS WITH THE TARGET ACTING AS A MEMORY WITH A REGISTER SIZE OF 4 BYTES

In [ ]:
MEMORY_ADDRESS  = 0x0000
DATA            = [i%0xFA for i in range(0,1024)]

request_result = supernova_target.i3cTargetWriteMemory(getId(), MEMORY_ADDRESS, DATA)

##### 3.13.1 I3C TARGET WRITE MEMORY

The public method ``Supernova.i3cTargetWriteMemory()`` writes the memory of the target Supernova using only USB communication.

```python
i3cTargetWriteMemory(id, memoryAddr, data)
```

- ``id: c_uint16``: Identifies the command.
- ``memoryAddr: c_uint16``: Address of the memory to start writing.
- ``data: list``: Data to write.

In [ ]:
MEMORY_ADDRESS  = 0x0A
DATA            = [0xFF for i in range(0,8)]

request_result = supernova_target.i3cTargetWriteMemory(getId(), MEMORY_ADDRESS, DATA)

##### 3.13.2 I3C TARGET READ MEMORY

The public method ``Supernova.i3cTargetReadMemory()`` reads the memory of the target Supernova using only USB communication.

```python
i3cTargetReadMemory(id, memoryAddr, length)
```

- ``id: c_uint16``: Identifies the command.
- ``memoryAddr: c_uint16``: Address of the memory to start reading.
- ``length: c_uint16``: Data length the user intends to read.

In [ ]:
MEMORY_ADDRESS  = 0x00
DATA_LENGTH     = 1024 # in bytes

request_result = supernova_target.i3cTargetReadMemory(getId(), MEMORY_ADDRESS, DATA_LENGTH)

#### 3.14 I3C WRITE AND READ

##### 3.14.1 I3C WRITE INDICATING THE START ADDRESS AND THE DATA

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x3B] # register 59 = byte 236
DATA            = [0xEE for i in range(8)]

request_result = supernova_controller.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

##### 3.14.2 I3C READ WITHOUT INDICATING THE START ADDRESS

The target will assume that the start of the new operation is the end of the previous one

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = []
LENGTH          = 4

request_result = supernova_controller.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

##### 3.14.3 I3C WRITE-READ

A write followed by a read with a RS in between

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x7D] # register 125 = byte 500
LENGTH          = 8 # in bytes

request_result = supernova_controller.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

##### 3.14.4 I3C WRITE INDICATING THE START ADDRESS FOLLOWED BY A READ WITHOUT ADDRESS

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0xFE] # register 254 = byte 1016  
DATA            = []

request_result = supernova_controller.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = []
LENGTH          = 4

request_result = supernova_controller.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

#### 3.14.5 BORDER CASES

##### 3.14.5.1 THE TRANSFER STARTS IN AN ALLOWED ADDRESS BUT TRIES TO SURPASS THE MEMORY RANGE ON THE GO

It will only modify the bytes in the allowed range and discard the rest. The end of the transfer is taken as the end of the memory

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0xFD] # register 253 = byte 1012
DATA            = []

request_result = supernova_controller.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = []
LENGTH          = 40

request_result = supernova_controller.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

#### 3.15 LONG TRANSFERS

##### 3.15.1 I3C WRITE INDICATING THE START ADDRESS AND THE DATA

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x00] 
DATA            = [0x47 for i in range(254)] + [0x07] + [0x37 for i in range(769)]

request_result = supernova_controller.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

##### 3.15.2 I3C WRITE-READ

A write followed by a read with a RS in between

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x00] 
LENGTH          = 255 # in bytes

request_result = supernova_controller.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

##### 3.15.3 I3C WRITE INDICATING THE START ADDRESS FOLLOWED BY A READ WITHOUT ADDRESS

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x00] 
DATA            = []

request_result = supernova_controller.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x00] 
LENGTH          = 255 # in bytes

request_result = supernova_controller.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)

##### 3.15.3 I3C WRITE INDICATING THE START ADDRESS FOLLOWED BY A READ WITHOUT ADDRESS

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = [0x00] 
DATA            = []

request_result = supernova_controller.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

In [ ]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_100_KHZ
REG_ADDR        = []
LENGTH          = 255

request_result = supernova_controller.i3cRead(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    LENGTH)